In [1]:
import pandas as pd
from glob import glob

In [2]:
data = glob('*.xlsx')

In [3]:
data

['5_문어체_조례.xlsx',
 '2_대화체.xlsx',
 '1_구어체(2).xlsx',
 '1_구어체(1).xlsx',
 '3_문어체_뉴스(2).xlsx',
 '3_문어체_뉴스(3).xlsx',
 '3_문어체_뉴스(1).xlsx',
 '4_문어체_한국문화.xlsx',
 '3_문어체_뉴스(4).xlsx',
 '6_문어체_지자체웹사이트.xlsx']

In [4]:
pd.read_excel('2_대화체.xlsx')

,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,비즈니스,회의,의견 교환하기,1,B-1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,비즈니스,회의,의견 교환하기,1,A-2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,회의,의견 교환하기,1,B-2,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,비즈니스,회의,의견 교환하기,2,A-1,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...,...,...,...,...,...
99995,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",24999,B-2,"저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.",It seems that we didn't place the price tags c...
99996,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-1,"백화점 포인트로 계산하고 싶은데, 가능한가요?",Can I pay using the department store points?
99997,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,B-1,"네, 물론이죠, 전화번호 입력해주시면 됩니다.","Yes, of course, you just need to enter your ph..."
99998,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-2,"입력했어요, 전액 백화점 포인트로 결제하고 싶어요.","I entered it, I want to pay it with all the de..."


In [5]:
df = pd.DataFrame(columns = ['원문','번역문'])

file_list = [ '2_대화체.xlsx',
 '1_구어체(2).xlsx',
 '1_구어체(1).xlsx',
 '3_문어체_뉴스(2).xlsx',
 '3_문어체_뉴스(3).xlsx',
 '3_문어체_뉴스(1).xlsx',
 '4_문어체_한국문화.xlsx',
 '3_문어체_뉴스(4).xlsx']

for data in file_list:
    temp = pd.read_excel(data)
    df = pd.concat([df,temp[['원문','번역문']]])
    

In [6]:
df.head()

,원문,번역문
0,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import TranslationDataset, Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

## 토크나이저 만들기

In [8]:
from torchtext.legacy import data 
from konlpy.tag import Okt

tokenizer = Okt()

In [9]:
def tokenize_kor(text):
    """한국어를 tokenizer해서 단어들을 리스트로 만든 후 reverse"""
    return [text_ for text_ in tokenizer.morphs(text)][::-1]

def tokenize_en(text):
    """영어를 split tokenizer해서 단어들을 리스트로 만듦"""
    return [text_ for text_ in text.split()]

# 필드 정의

SRC = data.Field(tokenize = tokenize_kor,
                init_token = '<sos>',
                eos_token = '<eos>')

TRG = data.Field(tokenize = tokenize_en,
                init_token = '<sos>',
                eos_token = '<eos>',
                lower = True)


# 데이터셋 만들기 (전체 데이터중 100,000개만 사용)


In [10]:
df

,원문,번역문
0,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...
199995,복무기간 단축안은 10월 전역자부터 2주 단위로 하루씩 단축해 육군·해병대·의무경찰...,The proposed reduction of the service period w...
199996,"실제로 이번 인사에서 인천지검 특수부장, 서울중앙지검 증권범죄합동수사단장 등을 지낸...","In fact, the vice chief of the Seoul Eastern D..."
199997,29일 서울 서초구 한국산업기술진흥협회 중회의실에서 열린 ‘이공계 우수인재 양성 및...,"On the 29th, at a meeting of experts on ""how t..."
199998,광주시교육청은 “지난 1일과 2일 한유총 광주지회로부터 장휘국 교육감 면담 요청이 ...,"The Gwangju Office of Education said, “There w..."


In [11]:
df_shuffled=df.sample(frac=1).reset_index(drop=True)

In [13]:
from sklearn.model_selection import KFold
df_ = df_shuffled[:100000]
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

for i,(trn_idx,val_idx) in enumerate(kf.split(df_['원문'])):
    trn = df_.iloc[trn_idx]
    val = df_.iloc[val_idx]

In [14]:
print('trn size: ',len(trn))
print('val size: ',len(val))

trn size:  80000
val size:  20000


In [15]:
trn.to_csv('trn.csv',index = False)
val.to_csv('val.csv',index = False)

In [16]:
from torchtext.legacy.data import TabularDataset

train_data, validation_data =TabularDataset.splits(
     path='', train='trn.csv',validation= 'val.csv', format='csv',
        fields=[('원문', SRC), ('번역문', TRG)], skip_header=True)

In [ ]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('검증 샘플의 개수 : {}'.format(len(validation_data)))

### Vocab 

In [ ]:
SRC.build_vocab(train_data, min_freq = 2, max_size = 50000)
TRG.build_vocab(train_data, min_freq = 2, max_size = 50000)

### data loader 

In [19]:
from torchtext.legacy.data import Iterator

# 하이퍼파라미터
batch_size = 128
lr = 0.001
EPOCHS = 20

train_loader = Iterator(dataset = train_data, batch_size = batch_size)
val_loader = Iterator(dataset = validation_data, batch_size = batch_size)

In [20]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('검증 데이터의 미니 배치 수 : {}'.format(len(val_loader)))

훈련 데이터의 미니 배치 수 : 625
검증 데이터의 미니 배치 수 : 157


# 모델 설계

### encoder
- 2 layer RNN
- Layer 1: 독일어 토큰의 임베딩을 입력으로 받고 은닉상태 출력
- Layer 2 : Layer1의 은닉상태를 입력으로 받고 새로운 은닉상태 출력
- 각 layer마다 초기 은닉상태 h_0 필요 (0으로 초기화 ?)
- 각 layer마다 context vector 'z'를 출력

In [31]:
# encoder 



class Encoder(nn.Module):
    """seq2seq의 encoder


    input_dim : input 데이터의 vocab size 
    단어들의 index가 embedding 함수로 넘겨짐

    emb_dim : embedding layer의 차원
    embedding 함수 : one-hot vector를 emb_dim 길이의 dense vector로 변환

    hid_dim : 은닉 상태의 차원 ( = cell state의 차원)

    n_layers : RNN 안의 레이어 개수 (여기선 2개)

    dropout : 사용할 드롭아웃의 양 (오버피팅 방지하는 정규화 방법)


    """
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout = 0.2):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU( emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        
        #src = [src len, batch_size)]
        embedded = self.dropout(self.embedding(src))
        #embeded = [src len, batch size, emb dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        
        
        return hidden




### decoder
- Layer 1 : 직전 time-stamp로 부터 은닉 상태(s)와 cell state를 받고, 이들과 embedded token인 y_t를 입력으로 받아 새로운 은닉상태와 cell state를 만들어냄
- Layer 2 : Layer 2의 은닉 상태(s)와 Layer 2에서 직전 time-stamp의 은닉 상태(s)와 cell state를 입력으로 받아 새로운 은닉 상태와 cell state를 만들어냄
- Decoder Layer1의 `첫 은닉상태(s)와 cell state` = `context vector (z)` = `Encoder Layer 1의 마지막 은닉상태(h)와 cell state`
- Decoder RNN/LSTM의 맨 위 Layer의 은닉 상태를 Linear Layer인 f에 넘겨서 다음 토큰이 무엇일지 예측함

 

In [32]:
class Decoder(nn.Module) : 
    """seq2seq의 decoder


    input_dim : input 데이터의 vocab size 
    단어들의 index가 embedding 함수로 넘겨짐

    emb_dim : embedding layer의 차원
    embedding 함수 : one-hot vector를 emb_dim 길이의 dense vector로 변환

    hid_dim : 은닉 상태의 차원 ( = cell state의 차원)

    n_layers : RNN 안의 레이어 개수 (여기선 2개)

    dropout : 사용할 드롭아웃의 양 (오버피팅 방지하는 정규화 방법)


    """
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.emb_dim = emb_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        
        # Decoder에서 항상 n directions = 1
        # 따라서 hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]
        
        # input = [1, batch size]
        input = input.unsqueeze(0)
        
        # embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))
        
        output, hidden = self.rnn(embedded, hidden)
        
        # output = [seq len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        # Decoder에서 항상 seq len = n directions = 1 
        # 한 번에 한 토큰씩만 디코딩하므로 seq len = 1
        # 따라서 output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        
        # prediction = [batch size, output dim]
        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden
        


In [33]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # Encoder와 Decoder의 hidden dim이 같아야 함 
        assert encoder.hid_dim == decoder.hid_dim, "encoder와 decoder의 hidden dim이 다름."
        assert encoder.n_layers == decoder.n_layers, "encoder와 decoder의 n_layers이 다름."

    def forward(self, src, trg ,teacher_forcing_ratio = 0.5):

        # src = [src len, batch size]
        # trg = [trg len, batch size]

        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        # decoder 결과를 저장할 텐서
        outputs = torch.zeros([trg_len,batch_size , trg_vocab_size])

        # encoder의 마지막 은닉 상태가 Deocder의 초기 은닉상태로 쓰임
        hidden = self.encoder(src)

        # decoder에 들어갈 첫 input은 <sos>토큰
        input = trg[0,:]

        #target length만큼 반복
        # range(0,trg_len)이 아니라 range(1,trg_len)인 이유 : 0번째 trg는 항상 <sos>라서 그에 대한 output도 항상 0 
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            # 확률 가장 높게 예측한 토큰
            top1 = output.argmax(1)

            #teacher_force = 1 = true 면 trg[t]를 아니면 top1을 input으로 사용
            input = trg[t] if teacher_force else top1
        return outputs



In [34]:
device = torch.device('cuda:1')

In [35]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)

# Encoder embedding dim
enc_emb_dim = 256
# Decoder embedding dim
dec_emb_dim = 256

hid_dim=512
n_layers=2

enc_dropout = 0.5
dec_dropout=0.5

enc = Encoder(input_dim, enc_emb_dim, hid_dim, n_layers, enc_dropout)
dec = Decoder(output_dim, dec_emb_dim, hid_dim, n_layers, dec_dropout)

model = Seq2Seq(enc, dec, device).to(device)

In [36]:
optimizer = optim.Adam(model.parameters())

# <pad> 토큰의 index를 넘겨 받으면 오차 계산하지 않고 ignore하기
# <pad> = padding
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = trg_pad_idx)

In [37]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss=0
    
    for i, batch in enumerate(iterator):
        src = batch.원문.to(device)
        trg = batch.번역문.to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        
        # loss 함수는 2d input으로만 계산 가능 
        output = output[1:].view(-1, output_dim).to(device)
        trg = trg[1:].view(-1)
        
        # trg = [(trg len-1) * batch size]
        # output = [(trg len-1) * batch size, output dim)]
        loss = criterion(output, trg)
        
        loss.backward()
        
        # 기울기 폭발 막기 위해 clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss+=loss.item()
        
    return epoch_loss/len(iterator)

In [38]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.원문.to(device)
            trg = batch.번역문.to(device)
            
            # teacher_forcing_ratio = 0 (아무것도 알려주면 안 됨)
            output = model(src, trg, 0)
            
            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim).to(device)
            trg = trg[1:].view(-1)
            
            # trg = [(trg len - 1) * batch size]
            # output = [(trg len - 1) * batch size, output dim]
            
            loss = criterion(output, trg)
            
            epoch_loss+=loss.item()
        
        return epoch_loss/len(iterator)

In [39]:
# function to count training time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 92m 8s
	Train Loss: 7.001 | Train PPL: 1097.763
	 Val. Loss: 6.758 |  Val. PPL: 860.662
Epoch: 02 | Time: 91m 49s
	Train Loss: 6.448 | Train PPL: 631.544
	 Val. Loss: 6.604 |  Val. PPL: 737.715
Epoch: 03 | Time: 91m 46s
	Train Loss: 6.159 | Train PPL: 472.787
	 Val. Loss: 6.506 |  Val. PPL: 669.069
Epoch: 04 | Time: 92m 2s
	Train Loss: 5.947 | Train PPL: 382.529
	 Val. Loss: 6.448 |  Val. PPL: 631.391
Epoch: 05 | Time: 92m 4s
	Train Loss: 5.772 | Train PPL: 321.065
	 Val. Loss: 6.406 |  Val. PPL: 605.700
Epoch: 06 | Time: 92m 4s
	Train Loss: 5.617 | Train PPL: 274.975
	 Val. Loss: 6.392 |  Val. PPL: 597.030
Epoch: 07 | Time: 92m 19s
	Train Loss: 5.489 | Train PPL: 242.084
	 Val. Loss: 6.401 |  Val. PPL: 602.354
Epoch: 08 | Time: 92m 22s
	Train Loss: 5.358 | Train PPL: 212.199
	 Val. Loss: 6.385 |  Val. PPL: 592.977
Epoch: 09 | Time: 92m 39s
	Train Loss: 5.254 | Train PPL: 191.396
	 Val. Loss: 6.417 |  Val. PPL: 612.416
Epoch: 10 | Time: 93m 57s
	Train Loss: 5.154 | Tr

In [96]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    """
    문장을 받아 번역 해주는 함수.
    """
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        tokens = [text_ for text_ in tokenizer.morphs(sentence)][::-1]
    else:
        raise Exception("input 데이터가 str이 아닙니다.")
        

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        hidden = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden)

        pred_token = output.argmax(1).item()
        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        
    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [95]:
sentence = '나는 가고싶다 집으로'

translate_sentence(sentence, SRC, TRG, model, device, max_len=50)

전체 소스 토큰: ['<sos>', '으로', '집', '가고싶다', '는', '나', '<eos>']
소스 문장 인덱스: [2, 14, 209, 0, 13, 48, 3]
[2, 16, 71, 6, 116, 1582]


['i', 'like', 'to', 'go', 'home.']